In [1]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
# using CuArrays

# Fetch MNIST images

In [2]:
# Classify MNIST digits with a convolutional network
imgs = MNIST.images()
size(imgs)

(60000,)

In [3]:
MNIST.labels()'

1×60000 LinearAlgebra.Adjoint{Int64,Array{Int64,1}}:
 5  0  4  1  9  2  1  3  1  4  3  5  3  …  7  8  9  2  9  5  1  8  3  5  6  8

In [4]:
labels = onehotbatch(MNIST.labels(), 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  1  0  1  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  1  0  1     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  1  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  1  0  …  0  0  0  0  0  1  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  1  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  1  0  1  0  0  0  0  0  0  0

In [5]:
# Partition into batches of size 1,000
train = [(cat(
               float.(imgs[i])..., dims = 4), 
               labels[:,i]
             )
         for i in partition(1:60_000, 1000)
        ]
train[1][1]

28×28×1×1000 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0       0.0       …  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0       …  0.498039  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.25098   0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0       …  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0          0.0       0.0  0.0  0.0  0.0
 ⋮   

In [6]:
train = gpu.(train)

60-element Array{Tuple{Array{Float64,4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0

In [7]:
# Prepare test set (first 1,000 images)
tX = cat(float.(MNIST.images(:test)[1:1000])..., dims = 4) |> gpu

28×28×1×1000 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.329412     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.870588     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.262745     0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0
 ⋮   

In [8]:
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9) |> gpu

10×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  0  1  0  0  0  0  0  0  1  0  0  …  0  0  0  0  0  1  0  0  0  1  0  0
 0  0  1  0  0  1  0  0  0  0  0  0  0     1  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  1  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  1  0  1  0  0  1     0  0  0  0  1  0  0  0  0  0  0  1

In [9]:
m = Chain(
  Conv((2,2), 1=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(288, 10), softmax) |> gpu

Chain(Conv((2, 2), 1=>16, relu), #5, Conv((2, 2), 16=>8, relu), #6, #7, Dense(288, 10), softmax)

In [10]:
m(train[1][1])

Tracked 10×1000 Array{Float32,2}:
 0.0472202  0.054291   0.0714605  …  0.0388284  0.0750903  0.0509503
 0.0709964  0.105972   0.071402      0.143572   0.0413494  0.0891363
 0.126917   0.0752191  0.129565      0.0648659  0.0734906  0.0566442
 0.0528754  0.0374397  0.0323719     0.0333266  0.0393383  0.0513794
 0.0945245  0.0965394  0.0864545     0.13217    0.0855065  0.102882
 0.202723   0.196748   0.167559   …  0.153014   0.233038   0.123006
 0.122126   0.150198   0.11679       0.148875   0.176246   0.143371
 0.0641994  0.113161   0.143304      0.130621   0.142139   0.216822
 0.120125   0.0928575  0.0835587     0.0923451  0.0624005  0.0842167
 0.0982934  0.0775742  0.0975339     0.0623826  0.0714012  0.0815925

In [11]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

In [12]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [13]:
evalcb = throttle(() -> @show(accuracy(tX, tY)), 10)

(::Flux.var"#throttled#14"{Flux.var"#throttled#10#15"{Bool,Bool,var"#11#12",Int64}}) (generic function with 1 method)

In [14]:
opt = ADAM(params(m))

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = ADAM(::Tracker.Params) at deprecations.jl:46
└ @ Flux.Optimise /Users/uki/.julia/packages/Flux/qXNjB/src/optimise/deprecations.jl:46


#24 (generic function with 1 method)

In [15]:
parameters = Flux.params(m)

Params([Float32[-0.01510764 -0.48567623; 0.016395932 0.27114445]

Float32[-0.34065667 -0.5275839; 0.4512665 0.16363308]

Float32[-0.13466696 0.35228187; -0.3405988 -0.41990605]

...

Float32[0.34221014 -0.38456997; -0.29182163 -0.49177065]

Float32[0.034284696 0.20368914; 0.07465275 -0.06454012]

Float32[0.16741422 -0.22917181; -0.21027678 0.19856708] (tracked), Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] (tracked), Float32[-0.028520653 0.32262337; 0.02053907 0.25718552]

Float32[-0.119390905 -0.26668596; -0.02827884 0.4095975]

Float32[0.22246404 -0.018762723; 0.14895827 -0.26772758]

...

Float32[0.14449649 0.25733793; 0.2156755 0.2733501]

Float32[-0.15994082 0.41189343; -0.23659733 0.12598762]

Float32[-0.086026385 0.4597123; 0.07805142 -0.23814489]

Float32[0.32468823 0.46123824; 0.13152756 0.43427992]

Float32[-0.23785219 0.4495066; -0.1488969 -0.22332422]

Float32[-0.19529846 0.22862501; -0.26934293 0.10540032]

...

Float32[-0.3281821

In [16]:
#@epochs 100 Flux.train!(loss, parameters, regData, opt)
Flux.train!(loss, parameters, train, opt, cb = evalcb)

MethodError: MethodError: no method matching ∇maxpool(::Array{Float32,4}, ::Array{Float64,4}, ::Array{Float64,4}, ::PoolDims{2,(2, 2),(2, 2),(0, 0, 0, 0),(1, 1)})
Closest candidates are:
  ∇maxpool(::AbstractArray{T,N}, !Matched::AbstractArray{T,N}, !Matched::AbstractArray{T,N}, ::PoolDims; kwargs...) where {T, N} at /Users/uki/.julia/packages/NNlib/FAI3o/src/pooling.jl:123